In [ ]:
from download_day import download_day
from load_saved_fd4 import load_saved_fd4
from scrape_aircraft_list import scrape_aircraft_list
from get_perf_model_typecodes import get_perf_model_typecodes 
from match_icao_model import match_icao_model
from process_airport_list import process_airport_list
from generate_flightpath import generate_flightpath
from plot_flightpaths import plot_flightpaths
from get_engine_data import get_engine_data
from perf_model_powerplant_parser import perf_model_powerplant_parser
from match_engine_to_emissions_db import match_engine_to_emissions_db
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from time import sleep
from geopy import distance
from icet import icet
from bffm2 import bffm2
%matplotlib inline

In [ ]:
# User Inputs:
start_time_str       = '2024-01-01T00:00:00Z'
stop_time_str        = '2024-12-31T23:59:59Z'
query_limit          = 30e4
send_notification    = True
make_plot            = False
output_dir           = "/scratch/omg28/Data/no_track2024"

# Convert start and stop times to datetime objects
start_time_simple = pd.to_datetime(start_time_str).strftime("%Y-%m-%d")
stop_time_simple = pd.to_datetime(stop_time_str).strftime("%Y-%m-%d")

Section 1. Download flight information from Opensky

In [ ]:
"""
# Download data from OpenSky history database
# WARNING: This may take a long time to run and will use a lot of disk space. Recommend running this in a .py script to avoid recurring memory issues found in Jupyter notebooks.
download_day(start_time_str, stop_time_str, query_limit, send_notification, make_plot, output_dir)
"""

In [ ]:
# load the saved day of data from a .pkl file to a pandas dataframe
loaded_day = load_saved_fd4(stop_time_str, output_dir, query_limit)

Section 2. Scrape the EUCONTROL database for aircraft flight performance information

In [ ]:
"""
# scrape the list of aircraft with performance models from the EUCONTROL website
perf_model_typecodes = get_perf_model_typecodes()
## WARNING: THIS CAN GET YOUR IP ADDRESS BLOCKED IF YOU RUN IT REPEATEDLY. IT IS A WEB SCRAPER.
aircraft_list = scrape_aircraft_list(perf_model_typecodes)

"""

In [ ]:
# remove the helicopters from the aircraft performance database list
perf_models = pd.read_csv("aircraft_performance_table.csv")
perf_models = perf_models[~perf_models['type'].str.contains('H')]
perf_models.to_csv("aircraft_performance_table_filtered.csv", index=False)
pd.to_pickle(perf_models, "aircraft_performance_table_filtered.pkl")


Section 3. Process the aircraft performance and flight data, retaining only flights we have takeoff, landing, plane type, and aircraft performance model information about. Further, drop all flights with aircraft codes that do not correspond to a code in the ourairports registry.

In [ ]:
# process the loaded time period of data. - 20s
# - matches 24 bit transponder codes from Feb 2025 aircraft metadata database with the ICAO typecode
# - removes all flights for which the transponder code does not have a corresponding ICAO typecode
# - removes all flights for which the takeoff OR landing airport is unknown
flights_with_typecode = match_icao_model(start_time_str, stop_time_str, query_limit)


In [ ]:
# Load in the list of ICAO typecodes that have a performance model
flights_with_typecode = pd.read_pickle('/scratch/omg28/Data/aircraftdb/2024-01-01_to_2024-12-31_300000.pkl')
# load in the list of all flights with typecode aand takeoff/landing airport from 2024
performance_model_typecodes = pd.read_pickle('/scratch/omg28/Data/aircraftdb/performance_models_typecodes.pkl')


# get info before the merge
print("Before merging:")
print(flights_with_typecode.info())

# inner join the loaded year of data with the scraped aircraft list
# - this will remove all flights for which the ICAO typecode does not have a corresponding performance model
flights_with_perf_typecode = pd.merge(flights_with_typecode, performance_model_typecodes, how='inner', on = 'typecode')

print("After merging:")
print(flights_with_perf_typecode.info())


In [ ]:
# print a list of typecodes in complete_flights_perf_model that do not have a performance model
missing_perf_models = flights_with_typecode[~flights_with_typecode['typecode'].isin(performance_model_typecodes['typecode'])]
print("Aircraft models with observed flights, but no performance model: " + str(len(missing_perf_models['typecode'].unique())))
print("total flights in 2024 with TOLD and typecode, but no performance model: " + str(len(missing_perf_models)))


In [ ]:
# print a list of typecodes in performance_model_typecodes that do not correspond to a flight
missing_flights = performance_model_typecodes[~performance_model_typecodes['typecode'].isin(flights_with_typecode['typecode'])]
print("Number of performance models with no flights: " + str(len(missing_flights['typecode'].unique())))
print("Number of flights with TOLD, typecode, and performance model: " + str(len(flights_with_perf_typecode)))


In [ ]:
# count all flights starting and ending at the same airport
num_flights_no_dist = len(flights_with_perf_typecode[flights_with_perf_typecode['estdepartureairport'] == flights_with_perf_typecode['estarrivalairport']])
print("Number of flights with no distance: " + str(num_flights_no_dist))

# remove all flights starting or ending at the same airport
flights_with_perf_typecode_dist = flights_with_perf_typecode[flights_with_perf_typecode['estdepartureairport'] != flights_with_perf_typecode['estarrivalairport']]
print("Number of flights with TOLD, typecode, performance model, and nonzero distance: " + str(len(flights_with_perf_typecode)))


In [ ]:
# load the takeoff and landing airport location information to the dataframe.
all_airports = process_airport_list()

# add the estimated departure airport location information to the dataframe
# - this will remove all flights for which the departure airport is unknown
flights_with_perf_typecode_dist_dep = pd.merge(flights_with_perf_typecode_dist, all_airports, how='inner', left_on = 'estdepartureairport', right_on = 'ident')
flights_with_perf_typecode_dist_dep.rename(columns = {'latitude_deg': 'estdeparturelat', 'longitude_deg': 'estdeparturelong', 'elevation_ft': 'estdeparturealt_ft'}, inplace=True)
flights_with_perf_typecode_dist_dep.drop(columns = ['ident', 'gps_code'], inplace=True)
# count the number of flights with no departure airport in the database
print("Number of flights with no departure airport location info in database: " + str(len(flights_with_perf_typecode_dist) - len(flights_with_perf_typecode_dist_dep)))

# add the estimated arrival airport location information to the dataframe
all_flights_filtered = pd.merge(flights_with_perf_typecode_dist_dep, all_airports, how='inner', left_on = 'estarrivalairport', right_on = 'ident')
all_flights_filtered.rename(columns = {'latitude_deg': 'estarrivallat', 'longitude_deg': 'estarrivallong', 'elevation_ft': 'estarrivalalt_ft'}, inplace=True)
all_flights_filtered.drop(columns = ['ident', 'gps_code', 'id_x', 'id_y'], inplace=True)
# count the number of flights with no arrival airport in the database
print("Number of flights with no arrival airport location info in database: " + str(len(flights_with_perf_typecode_dist_dep) - len(all_flights_filtered)))
print("Number of flights with TOLD, typecode, performance model, distance, and departure/arrival airport info: " + str(len(all_flights_filtered)))


Section 4. Using the great circle distance, calculate the distance between the airport takeoff and landing.

In [ ]:
# calculate the great circle distance between the departure and arrival airports - 4 min
all_flights_filtered['gc_km'] = all_flights_filtered.apply(lambda x: distance.great_circle((x['estdeparturelat'], x['estdeparturelong']), (x['estarrivallat'], x['estarrivallong'])).km, axis=1)
# adjust great circle distance using eq. 6 of seymour, et al (2020)
# all_flights_filtered = pd.read_pickle(f'/scratch/omg28/Data/no_track_2024/{start_time_simple}_to_{stop_time_simple}_filtered.pkl')

In [ ]:
# could be sped up by using a vectorized function - 1 min
all_flights_filtered['gc_FEAT_km'] = all_flights_filtered.apply(lambda x: 1.0387 * x['gc_km'] + 40.5, axis = 1)

In [ ]:
# save the merged dataframe to a .pkl file - 15s
all_flights_filtered.to_pickle(f'/scratch/omg28/Data/no_track2024/{start_time_simple}_to_{stop_time_simple}_filtered.pkl')

In [ ]:
print(all_flights_filtered.info())
print(all_flights_filtered.head())

In [ ]:
# remove all flights under 200 km:
all_flights_filtered = all_flights_filtered[all_flights_filtered['gc_FEAT_km'] > 200]
print("Number of flights with TOLD, typecode, performance model, distance, and departure/arrival airport info over 200 km: " + str(len(all_flights_filtered)))

Section 5: parse the engine data and emissions data and add it to the aircraft performance model.

In [ ]:
# Loads data from the saved engine data .csv files for analysis. Saves this dataframe to a .pkl file
# Setting "remove_superseded" to True will remove all engines that are superseded by an updated emissions model
engine_models = get_engine_data(update_superseded=False)
engine_models_removed = get_engine_data(update_superseded=True)
# calculate how many engines were removed
print("Number of engines that would be removed: " + str(len(engine_models['Engine Identification'].unique()) - len(engine_models_removed['Engine Identification'].unique())))
# print the list of the engine models that are in engine_models_kept but not in engine_models_removed
superseded_not_replaced = engine_models[~engine_models['Engine Identification'].isin(engine_models_removed['Engine Identification'])].get('Engine Identification').unique()
print("List of engines that were removed: " + str(superseded_not_replaced))
# POTENTIAL FIX: REMOVE ENGINES THAT HAVE BEEN SUPERSEDED BY AN UPDATED EMISSIONS MODE


In [ ]:
# loads the aircraft performance model data from the saved .csv file
performance_models = pd.read_csv("aircraft_performance_table_filtered.csv")

In [ ]:
# extracts the powerplant data from the performance model dataframe and appends it to new columns in the performance model dataframe
allowed_manufacturers = engine_models['Manufacturer'].unique().tolist()
performance_models = perf_model_powerplant_parser(performance_models, coerce_manufacturer=True, allowed_manufacturers=allowed_manufacturers)


In [ ]:
# attempts to match the powerplants extracted from the EUCONTROL aircraft performance model with an engine model in the engine emissions database
# reset the index of engine_models so that 'UID No' becomes a column rather than the index, otherwise the merge will not work
engine_models.reset_index(inplace=True)
# attempts to match the powerplants extracted from the EUCONTROL aircraft performance model with an engine model in the engine emissions database
performance_models_matched = match_engine_to_emissions_db(performance_models, engine_models)

In [ ]:
# from all rows without a matched engine model
performance_models_matched = performance_models_matched.dropna(subset=['matched_engine_id'])

In [ ]:
# print number of performance models whose engines match superseded engines
print("Number of performance models with engines that are superseded: " + str(len(performance_models_matched[performance_models_matched['matched_engine_id'].isin(superseded_not_replaced)])))
print("Number of performance models with engines that are NOT superseded: " + str(len(performance_models_matched[~performance_models_matched['matched_engine_id'].isin(superseded_not_replaced)])))
print("aircraft performance models with engines that are superseded: \n" + str(performance_models_matched[performance_models_matched['matched_engine_id'].isin(superseded_not_replaced)].get('typecode')))

In [ ]:
# Join the aircraft performance model data with the engine emissions database data
performance_and_emissions_model = pd.merge(performance_models_matched, engine_models, how='inner', left_on='matched_engine_uid_no', right_on='UID No')
pd.to_pickle(performance_and_emissions_model, "performance_and_emissions_model.pkl")

Section 6: generate and visualize flightpaths

In [ ]:
# performance_data=pd.read_pickle('aircraft_performance_data_table.pkl')
aircraft_typecode = 'A320'
flightpath = generate_flightpath(aircraft_typecode, 500, performance_and_emissions_model)

In [ ]:
import matplotlib.pyplot as plt
# Sum durations from the flightpath dictionary for each phase
climb_time = (flightpath['climb'].get('t_climb_0_5', 0) +
              flightpath['climb'].get('t_climb_5_10', 0) +
              flightpath['climb'].get('t_climb_10_15', 0) +
              flightpath['climb'].get('t_climb_15_24', 0) +
              flightpath['climb'].get('t_climb_ceil', 0))

descent_time = (flightpath['descent'].get('t_descent_5_0', 0) +
                flightpath['descent'].get('t_descent_10_5', 0) +
                flightpath['descent'].get('t_descent_15_10', 0) +
                flightpath['descent'].get('t_descent_24_15', 0) +
                flightpath['descent'].get('t_descent_ceil', 0))

cruise_time = flightpath['cruise'].get('t_cruise', 0)

# Prepare data for the barchart
phases = ['Climb', 'Cruise', 'Descent']
durations = [climb_time, cruise_time, descent_time]

# Optionally convert seconds to minutes (or leave in seconds)
# durations = [t / 60.0 for t in durations]

plt.figure()
bars = plt.bar(phases, durations, color=['skyblue', 'lightgreen', 'salmon'])
plt.xlabel('Flight Phases')
plt.ylabel('Duration (seconds)')
plt.title(f'Duration of Each Flight Leg ({aircraft_typecode})')

# Annotate bars with duration values
for bar, duration in zip(bars, durations):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height,
             f'{duration:.0f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()


In [ ]:

# Break down the climb phases
climb_labels = ['Climb 0-5', 'Climb 5-10', 'Climb 10-15', 'Climb 15-24', 'Climb Ceil']
climb_times = [
    flightpath['climb'].get('t_climb_0_5', 0),
    flightpath['climb'].get('t_climb_5_10', 0),
    flightpath['climb'].get('t_climb_10_15', 0),
    flightpath['climb'].get('t_climb_15_24', 0),
    flightpath['climb'].get('t_climb_ceil', 0)
]

# Break down the descent phases
descent_labels = ['Descent 5-0', 'Descent 10-5', 'Descent 15-10', 'Descent 24-15', 'Descent Ceil']
descent_times = [
    flightpath['descent'].get('t_descent_5_0', 0),
    flightpath['descent'].get('t_descent_10_5', 0),
    flightpath['descent'].get('t_descent_15_10', 0),
    flightpath['descent'].get('t_descent_24_15', 0),
    flightpath['descent'].get('t_descent_ceil', 0)
]

import matplotlib.pyplot as plt

plt.figure(figsize=(14, 6))

# Climb segments chart
plt.subplot(1, 2, 1)
bars_climb = plt.bar(climb_labels, climb_times, color='skyblue')
plt.xlabel('Climb Segments')
plt.ylabel('Time (seconds)')
plt.title(f'Climb Phase Breakdown ({aircraft_typecode})')
for bar, t in zip(bars_climb, climb_times):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{t:.1f}', ha='center', va='bottom')

# Descent segments chart
plt.subplot(1, 2, 2)
bars_descent = plt.bar(descent_labels, descent_times, color='salmon')
plt.xlabel('Descent Segments')
plt.ylabel('Time (seconds)')
plt.title(f'Descent Phase Breakdown ({aircraft_typecode})')
for bar, t in zip(bars_descent, descent_times):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{t:.1f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

import matplotlib.pyplot as plt

# Overall phase distances (in meters)
climb_distance = (np.atleast_1d(flightpath['climb'].get('s_climb_0_5', 0))[0] +
                  np.atleast_1d(flightpath['climb'].get('s_climb_5_10', 0))[0] +
                  np.atleast_1d(flightpath['climb'].get('s_climb_10_15', 0))[0] +
                  np.atleast_1d(flightpath['climb'].get('s_climb_15_24', 0))[0] +
                  np.atleast_1d(flightpath['climb'].get('s_climb_ceil', 0))[0])

cruise_distance = np.atleast_1d(flightpath['cruise'].get('s_cruise', 0))[0]

descent_distance = (np.atleast_1d(flightpath['descent'].get('s_descent_5_0', 0))[0] +
                    np.atleast_1d(flightpath['descent'].get('s_descent_10_5', 0))[0] +
                    np.atleast_1d(flightpath['descent'].get('s_descent_15_10', 0))[0] +
                    np.atleast_1d(flightpath['descent'].get('s_descent_24_15', 0))[0] +
                    np.atleast_1d(flightpath['descent'].get('s_descent_ceil', 0))[0])

phases = ['Climb', 'Cruise', 'Descent']
distances = [climb_distance, cruise_distance, descent_distance]

plt.figure(figsize=(8, 6))
overall_bars = plt.bar(phases, distances, color=['skyblue', 'lightgreen', 'salmon'])
plt.xlabel('Flight Phases')
plt.ylabel('Distance (meters)')
plt.title(f'Distance Covered in Each Flight Phase ({aircraft_typecode})')

# Annotate overall bars with distance values
for bar, dist in zip(overall_bars, distances):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height, f'{dist:,.0f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()


In [ ]:


# Breakdown of distances for each segment

# For Climb Phase
climb_keys = ['s_climb_0_5', 's_climb_5_10', 's_climb_10_15', 's_climb_15_24', 's_climb_ceil']
climb_distances = []
for key in climb_keys:
    value = flightpath['climb'].get(key, np.array([0]))
    # value is expected to be a numpy array; extract its first element if so
    if isinstance(value, np.ndarray):
        climb_distances.append(value[0])
    else:
        climb_distances.append(value)

# For Descent Phase
descent_keys = ['s_descent_5_0', 's_descent_10_5', 's_descent_15_10', 's_descent_24_15', 's_descent_ceil']
descent_distances = []
for key in descent_keys:
    value = flightpath['descent'].get(key, np.array([0]))
    if isinstance(value, np.ndarray):
        descent_distances.append(value[0])
    else:
        descent_distances.append(value)

plt.figure(figsize=(14, 6))

# Climb segments chart
plt.subplot(1, 2, 1)
bars_climb = plt.bar(climb_labels, climb_distances, color='skyblue')
plt.xlabel('Climb Segments')
plt.ylabel('Distance (meters)')
plt.title(f'Climb Phase Distance Breakdown ({aircraft_typecode})')
for bar, d in zip(bars_climb, climb_distances):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{d:,.0f}', ha='center', va='bottom')

# Descent segments chart
plt.subplot(1, 2, 2)
bars_descent = plt.bar(descent_labels, descent_distances, color='salmon')
plt.xlabel('Descent Segments')
plt.ylabel('Distance (meters)')
plt.title(f'Descent Phase Distance Breakdown ({aircraft_typecode})')
for bar, d in zip(bars_descent, descent_distances):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{d:,.0f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()


In [ ]:
# Build cumulative distance and altitude lists.
# Starting at runway: (distance, altitude) = (0, 0)
distances_list = [0]
altitudes_list = [0]

# Climb phase:
# First climb segment (0-5):
cum_distance = distances_list[-1]
for key in flightpath:
    for key2 in flightpath[key]:
        if key2.startswith('s_'):
            cum_distance += flightpath[key][key2]
            distances_list.append(cum_distance)
            altitudes_list.append(flightpath[key][f'{key2.replace('s_', 'h_')}_end'])

# Plot the altitude profile vs. cumulative flight distance.
plt.figure(figsize=(10, 6))
plt.plot(distances_list, altitudes_list, marker='o', linestyle='-')
plt.xlabel('Cumulative Flight Distance (meters)')
plt.ylabel('Flight Altitude (meters)')
plt.title(f'Flight Altitude vs. Flight Distance ({aircraft_typecode})')
plt.grid(True)
plt.show()

In [ ]:
# Generate 25 flight distances (km) linearly spaced from 200 to 5000 km.
num_flightpaths = 25
flight_distances = np.linspace(200, 1000, num_flightpaths)
flightpaths = [0] * num_flightpaths
 
# Loop through each flight distance and generate the flightpath.
for ii in np.arange(num_flightpaths):
    d_km = flight_distances[ii] 
    # Generate flightpath for a given flight distance (in km)
    fp = generate_flightpath(aircraft_typecode, d_km, performance_and_emissions_model)
    flightpaths[ii] = fp
plot_flightpaths(flightpaths)

### Visualize NOx Emissions by Flight Phase

In [ ]:
# Visualize NOx emissions for each climb and descent phase
import matplotlib.pyplot as plt

climb_nox_labels = ['Climb 0-5', 'Climb 5-10', 'Climb 10-15', 'Climb 15-24', 'Climb Ceil']
climb_nox_keys = ['NOx_climb_0_5', 'NOx_climb_5_10', 'NOx_climb_10_15', 'NOx_climb_15_24', 'NOx_climb_ceil']
climb_nox_values = [flightpath['climb'].get(k, 0) for k in climb_nox_keys]

descent_nox_labels = ['Descent Ceil', 'Descent 24-15', 'Descent 15-10', 'Descent 10-5', 'Descent 5-0']
descent_nox_keys = ['NOx_descent_ceil', 'NOx_descent_24_15', 'NOx_descent_15_10', 'NOx_descent_10_5', 'NOx_descent_5_0']
descent_nox_values = [flightpath['descent'].get(k, 0) for k in descent_nox_keys]

plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
bars_climb = plt.bar(climb_nox_labels, climb_nox_values, color='deepskyblue')
plt.xlabel('Climb Segments')
plt.ylabel('NOx Emissions (g)')
plt.title(f'NOx Emissions During Climb ({aircraft_typecode})')
for bar, val in zip(bars_climb, climb_nox_values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{val:.1f}', ha='center', va='bottom')

plt.subplot(1, 2, 2)
bars_descent = plt.bar(descent_nox_labels, descent_nox_values, color='orange')
plt.xlabel('Descent Segments')
plt.ylabel('NOx Emissions (g)')
plt.title(f'NOx Emissions During Descent ({aircraft_typecode})')
for bar, val in zip(bars_descent, descent_nox_values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{val:.1f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Cumulative NOx Emissions vs. Flight Distance
num_flightpaths = 25
flight_distances = np.linspace(200, 5000, num_flightpaths)
cumulative_nox = []
for d_km in flight_distances:
    fp = generate_flightpath(aircraft_typecode, d_km, performance_and_emissions_model)
    total_nox = 0
    for k in ['NOx_climb_0_5', 'NOx_climb_5_10', 'NOx_climb_10_15', 'NOx_climb_15_24', 'NOx_climb_ceil']:
        total_nox += fp['climb'].get(k, 0)
    for k in ['NOx_descent_ceil', 'NOx_descent_24_15', 'NOx_descent_15_10', 'NOx_descent_10_5', 'NOx_descent_5_0']:
        total_nox += fp['descent'].get(k, 0)
    total_nox += fp.get('cruise', {}).get('NOx_cruise', 0)
    cumulative_nox.append(total_nox)

plt.figure(figsize=(8, 6))
plt.plot(flight_distances, cumulative_nox, marker='o', color='purple')
plt.xlabel('Flight Distance (km)')
plt.ylabel('Cumulative NOx Emissions (g)')
plt.title(f'Cumulative NOx Emissions vs. Flight Distance ({aircraft_typecode})')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Instantaneous NOx flux vs. flight distance
import numpy as np
import matplotlib.pyplot as plt

# Build lists for distance and NOx flux (g/s) at each segment
segment_distances = [0]
segment_nox_flux = []

# Climb segments
climb_keys = [
    ('s_climb_0_5', 't_climb_0_5', 'NOx_climb_0_5'),
    ('s_climb_5_10', 't_climb_5_10', 'NOx_climb_5_10'),
    ('s_climb_10_15', 't_climb_10_15', 'NOx_climb_10_15'),
    ('s_climb_15_24', 't_climb_15_24', 'NOx_climb_15_24'),
    ('s_climb_ceil', 't_climb_ceil', 'NOx_climb_ceil'),
]
cum_dist = 0
for s_key, t_key, nox_key in climb_keys:
    s = flightpath['climb'].get(s_key, 0)
    t = flightpath['climb'].get(t_key, 0)
    nox = flightpath['climb'].get(nox_key, 0)
    if t > 0 and s > 0:
        cum_dist += s
        segment_distances.append(cum_dist)
        segment_nox_flux.append(nox / t)

# Cruise segment (if available)
s_cruise = flightpath['cruise'].get('s_cruise', 0)
t_cruise = flightpath['cruise'].get('t_cruise', 0)
nox_cruise = flightpath['cruise'].get('NOx_cruise', 0)
if t_cruise > 0 and s_cruise > 0:
    cum_dist += s_cruise
    segment_distances.append(cum_dist)
    segment_nox_flux.append(nox_cruise / t_cruise if t_cruise > 0 else 0)

# Descent segments
descent_keys = [
    ('s_descent_ceil', 't_descent_ceil', 'NOx_descent_ceil'),
    ('s_descent_24_15', 't_descent_24_15', 'NOx_descent_24_15'),
    ('s_descent_15_10', 't_descent_15_10', 'NOx_descent_15_10'),
    ('s_descent_10_5', 't_descent_10_5', 'NOx_descent_10_5'),
    ('s_descent_5_0', 't_descent_5_0', 'NOx_descent_5_0'),
]
for s_key, t_key, nox_key in descent_keys:
    s = flightpath['descent'].get(s_key, 0)
    t = flightpath['descent'].get(t_key, 0)
    nox = flightpath['descent'].get(nox_key, 0)
    if t > 0 and s > 0:
        cum_dist += s
        segment_distances.append(cum_dist)
        segment_nox_flux.append(nox / t)

# Ensure x and y have the same length by removing the initial 0 from segment_distances.
segment_distances = segment_distances[1:]

plt.figure(figsize=(10, 6))
plt.step(segment_distances, segment_nox_flux, where='post', marker='o')
plt.xlabel('Cumulative Flight Distance (meters)')
plt.ylabel('Instantaneous NOx Flux (g/s)')
plt.title(f'Instantaneous NOx Flux vs. Flight Distance ({aircraft_typecode})')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Instantaneous NOx flux and altitude vs. flight distance
import numpy as np
import matplotlib.pyplot as plt

# Build lists for distance, NOx flux (g/s), and altitude at each segment
segment_distances = [0]
segment_nox_flux = []
segment_altitudes = [0]

# Climb segments
climb_keys = [
    ('s_climb_0_5', 't_climb_0_5', 'NOx_climb_0_5', 'h_climb_0_5_end'),
    ('s_climb_5_10', 't_climb_5_10', 'NOx_climb_5_10', 'h_climb_5_10_end'),
    ('s_climb_10_15', 't_climb_10_15', 'NOx_climb_10_15', 'h_climb_10_15_end'),
    ('s_climb_15_24', 't_climb_15_24', 'NOx_climb_15_24', 'h_climb_15_24_end'),
    ('s_climb_ceil', 't_climb_ceil', 'NOx_climb_ceil', 'h_climb_ceil_end'),
]
cum_dist = 0
for s_key, t_key, nox_key, h_key in climb_keys:
    s = flightpath['climb'].get(s_key, 0)
    t = flightpath['climb'].get(t_key, 0)
    nox = flightpath['climb'].get(nox_key, 0)
    h = flightpath['climb'].get(h_key, 0)
    if t > 0 and s > 0:
        cum_dist += s
        segment_distances.append(cum_dist)
        segment_nox_flux.append(nox / t)
        segment_altitudes.append(h)

# Cruise segment (if available)
s_cruise = flightpath['cruise'].get('s_cruise', 0)
t_cruise = flightpath['cruise'].get('t_cruise', 0)
nox_cruise = flightpath['cruise'].get('NOx_cruise', 0)
h_cruise = flightpath['cruise'].get('h_cruise_end', 0)
if t_cruise > 0 and s_cruise > 0:
    cum_dist += s_cruise
    segment_distances.append(cum_dist)
    segment_nox_flux.append(nox_cruise / t_cruise if t_cruise > 0 else 0)
    segment_altitudes.append(h_cruise)

# Descent segments
descent_keys = [
    ('s_descent_ceil', 't_descent_ceil', 'NOx_descent_ceil', 'h_descent_ceil_end'),
    ('s_descent_24_15', 't_descent_24_15', 'NOx_descent_24_15', 'h_descent_24_15_end'),
    ('s_descent_15_10', 't_descent_15_10', 'NOx_descent_15_10', 'h_descent_15_10_end'),
    ('s_descent_10_5', 't_descent_10_5', 'NOx_descent_10_5', 'h_descent_10_5_end'),
    ('s_descent_5_0', 't_descent_5_0', 'NOx_descent_5_0', 'h_descent_5_0_end'),
]
for s_key, t_key, nox_key, h_key in descent_keys:
    s = flightpath['descent'].get(s_key, 0)
    t = flightpath['descent'].get(t_key, 0)
    nox = flightpath['descent'].get(nox_key, 0)
    h = flightpath['descent'].get(h_key, 0)
    if t > 0 and s > 0:
        cum_dist += s
        segment_distances.append(cum_dist)
        segment_nox_flux.append(nox / t)
        segment_altitudes.append(h)

# Ensure all arrays have the same length for plotting
min_len = min(len(segment_distances), len(segment_nox_flux), len(segment_altitudes))
segment_distances_plot = segment_distances[:min_len]
segment_nox_flux_plot = segment_nox_flux[:min_len]
segment_altitudes_plot = segment_altitudes[:min_len]

fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.step(segment_distances_plot, segment_nox_flux_plot, where='post', marker='o', color='tab:blue', label='NOx Flux (g/s)')
ax1.set_xlabel('Cumulative Flight Distance (meters)')
ax1.set_ylabel('Instantaneous NOx Flux (g/s)', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

ax2 = ax1.twinx()
ax2.plot(segment_distances_plot, segment_altitudes_plot, marker='s', color='tab:red', label='Altitude (m)')
ax2.set_ylabel('Altitude (meters)', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

fig.suptitle(f'Instantaneous NOx Flux and Altitude vs. Flight Distance ({aircraft_typecode})')
fig.tight_layout()
plt.show()

TEST CODE

In [17]:
from download_day import download_day
from load_saved_fd4 import load_saved_fd4
from scrape_aircraft_list import scrape_aircraft_list
from get_perf_model_typecodes import get_perf_model_typecodes 
from match_icao_model import match_icao_model
from process_airport_list import process_airport_list
from generate_flightpath import generate_flightpath
from plot_flightpaths import plot_flightpaths
from get_engine_data import get_engine_data
from perf_model_powerplant_parser import perf_model_powerplant_parser
from match_engine_to_emissions_db import match_engine_to_emissions_db
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from time import sleep
from geopy import distance
from icet import icet
# from bffm2 import bffm2
%matplotlib inline

In [ ]:
from ambiance import Atmosphere
import numpy as np
from pa_to_psi import pa_to_psi
from ft_to_m import ft_to_m
def bffm2(aircraft_data, altitude, mach, W_f):
    '''
    BFFM2 - Boeing Fuel Flow Model 2 is the second version of the Boeing Fuel Flow Model, which is a fuel flow model used to estimate the fuel consumption of commercial aircraft during flight. It is based on a combination of empirical data and theoretical models, and it takes into account various factors such as aircraft weight, altitude, speed, and engine performance.
    The BFFM2 model is used by airlines and aviation authorities to estimate fuel consumption and emissions for different flight profiles, and it is often used in conjunction with other models to provide a comprehensive analysis of aircraft performance.
    The BFFM2 model is a widely used tool in the aviation industry for fuel consumption and emissions estimation, and it is considered to be one of the most accurate models available for this purpose.
    The BFFM2 model is a proprietary model developed by Boeing, and it is based on extensive data collected from commercial aircraft operations. The model uses a combination of empirical data and theoretical models to estimate fuel consumption and emissions for different flight profiles, and it takes into account various factors such as aircraft weight, altitude, speed, and engine performance.
    '''
    # get the necessary data from the aircraft_data dictionary
    fuel_flow_icao = np.array([aircraft_data.iloc[0]['Fuel Flow T/O (kg/sec)'],
                               aircraft_data.iloc[0]['Fuel Flow C/O (kg/sec)'],
                               aircraft_data.iloc[0]['Fuel Flow App (kg/sec)'],
                               aircraft_data.iloc[0]['Fuel Flow Idle (kg/sec)']])
    ###################################################
    fuel_flow_icao = np.array([3.91, 3.10, 1.00, 0.30])
    ###################################################                        

    HC_ei_vec = np.array([aircraft_data.iloc[0]['HC EI T/O (g/kg)'],
                          aircraft_data.iloc[0]['HC EI C/O (g/kg)'],
                          aircraft_data.iloc[0]['HC EI App (g/kg)'],
                          aircraft_data.iloc[0]['HC EI Idle (g/kg)']])

    CO_ei_vec = np.array([aircraft_data.iloc[0]['CO EI T/O (g/kg)'],
                          aircraft_data.iloc[0]['CO EI C/O (g/kg)'],
                          aircraft_data.iloc[0]['CO EI App (g/kg)'],
                          aircraft_data.iloc[0]['CO EI Idle (g/kg)']])
    
    CO_ei_vec = np.array([0.24, 0.24, 0.24, 0.24]) # for testing purposes

    NOx_ei_vec = np.array([aircraft_data.iloc[0]['NOx EI T/O (g/kg)'],
                           aircraft_data.iloc[0]['NOx EI C/O (g/kg)'],
                           aircraft_data.iloc[0]['NOx EI App (g/kg)'],
                           aircraft_data.iloc[0]['NOx EI Idle (g/kg)']])

    NOx_ei_vec = np.array([19.43, 19.43, 19.43, 19.43]) # for testing purposes

    atmosphere = Atmosphere(altitude)
    P_amb = pa_to_psi(atmosphere.pressure)
    T_amb = Atmosphere.T2t(atmosphere.temperature)

    # 1. Calculate the ambient pressure and temperature at the given altitude:

    # Convert from kg/s to lbs/hr:
    fuel_flow_icao = fuel_flow_icao * 7936.64
    # 1. Installation Correction Factor (ICF) for the engine
    #                   T/O   C/O    App    Idle                
    icf_vec = np.array([1.01, 1.013, 1.020, 1.100])
    fuel_flow_icao = np.array(fuel_flow_icao * icf_vec)
    print(fuel_flow_icao)
    
    # transform all to log space:
    fuel_flow_icao = np.log(fuel_flow_icao)
    HC_ei_vec = np.log(HC_ei_vec)
    CO_ei_vec = np.log(CO_ei_vec)
    NOx_ei_vec = np.log(NOx_ei_vec)
    fuel_flow_evaluate = np.log(W_f)

    # 2. Use linear log-log fit on the emission index vs fuel flow rate
    def piecewise_fit(x, EI_vec, fuel_flow_icao_local):
        # Line between Idle and Approach:
        m_3_2 = (EI_vec[3] - EI_vec[2]) / (fuel_flow_icao_local[3] - fuel_flow_icao_local[2])
        b_3_2 = EI_vec[3] - m_3_2 * fuel_flow_icao_local[3]

        # Line at the average of T/0 and C/O:
        m_0_1 = 0
        b_0_1 = (EI_vec[0] + EI_vec[1])/2
        
        # find line between intersection and constant
        intersect_x = (b_0_1 - b_3_2) / m_3_2 if m_3_2 != 0 else float('inf')           
        # If intersection is within [App, C/O]
        if fuel_flow_icao_local[2] <= intersect_x <= fuel_flow_icao_local[1]:
            if x >= intersect_x:
                return b_0_1
            else:
                return m_3_2 * x + b_3_2
        else: # if the intersection is outside of app, CO
            # find line between approach and average of climbout and takeoff:
            m_2_1 = (EI_vec[2] - b_0_1) / (fuel_flow_icao_local[2] - fuel_flow_icao_local[1])
            b_2_1 = EI_vec[2] - m_2_1 * fuel_flow_icao_local[2]
            if x >= fuel_flow_icao_local[1]: # if x is greater than climbout
                return b_0_1
            elif x <= fuel_flow_icao_local[2]: # if x is less than approach
                return m_3_2 * x + b_3_2
            else: # if x is between approach and climbout
                return m_2_1 * x + b_2_1
    # For NOx: Fit a piecewise continuous line through all four points
    # Create a piecewise linear point-to-point interpolator in log-log space
    EI_HC_ret = np.exp(piecewise_fit(W_f, HC_ei_vec, fuel_flow_icao))
    EI_CO_ret = np.exp(piecewise_fit(W_f, CO_ei_vec, fuel_flow_icao))
    EI_NOx_ret = np.exp(np.interp(W_f, np.flip(fuel_flow_icao), np.flip(NOx_ei_vec)))
    # Calculate the interpolated emission indices for the given fuel flow rate:
    ###
    # log_fuel_flow_evaluate = np.log10(fuel_flow_evaluate)
    ###
   
    
    # 2a. calculate the temp and pressure correction factors:
    
    
    del_amb = P_amb/14.696
    print(del_amb)
    theta_amb = (T_amb + 273.15)/288.15
    print(theta_amb)
    # 2b. fuel flow values are further modified by the ambient values:
    W_ff = (W_f/del_amb) * (theta_amb**3.8) * np.exp(0.2*mach**2)
    print(W_ff)
    # 2c. calculate the humidity correction factor, using modified EUCONTROL correction (0.37802)
    phi = 0.6 # relative humidity, assumed to be 60% for this calculation
    beta = 7.90298*(1-373.16/(T_amb + 273.16)) + 3.00571 + (5.02808) * np.log(373.16/(T_amb + 273.16)) + 1.3816*10**(-7) * (1-10**(11.344*(1-(T_amb + 273.16)/373.16))) + 8.1328 * 10**(-3) * (10**(3.49149*(1-373.16/(T_amb+273.16)))-1)
    P_v = (0.014504)*10**beta
    omega = (0.62198*phi*P_v)/(P_amb-0.37802*phi*P_v)
    H_corr_factor = -19*(omega-0.0063)
    
    # 3. Compute the EI:
    rEI_HC = piecewise_fit(W_ff, HC_ei_vec, fuel_flow_icao)
    print(rEI_HC)
    rEI_CO = piecewise_fit(W_ff, CO_ei_vec, fuel_flow_icao)
    print(rEI_CO)
    rEI_NOx = np.interp(W_ff, np.flip(fuel_flow_icao), np.flip(NOx_ei_vec))
    
    EI_HC_corr = rEI_HC * theta_amb**3.3 / (del_amb**1.02)
    EI_CO_corr = rEI_CO * theta_amb**3.3 / (del_amb**1.02)
    EI_NOx_corr = rEI_NOx * (del_amb**1.02 * theta_amb**3.3)**0.5 * np.exp(H_corr_factor)   
    
    # 4. Calculate the final EI:
    return EI_HC_ret, EI_CO_ret, EI_NOx_ret


aircraft_data = pd.read_pickle('performance_and_emissions_model.pkl')

altitude = ft_to_m(39000)
mach = 0.84
W_f = 0.882

tester = aircraft_data[aircraft_data['typecode'] == 'B737']

ret = bffm2(tester, altitude, mach, W_f)
print(ret)

[31342.585024 24923.430592  8095.3728    2619.0912  ]
[0.19487933]
[0.75186535]
[1.7632858]
[5.75789313]
[-0.61978876]
(np.float64(8.53447641605149), np.float64(-0.619788758288394), np.float64(1.2884728005997825))
